In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sympy
from scipy.spatial.distance import cdist  # fast distance computation


In [2]:
wavelength = 1
Ns = 9
dy = wavelength / 2
Lz = 5 * wavelength
Nr = 9
k = 2 * np.pi / wavelength

In [3]:
def euclidean_distance(s, d) -> float:
    # s -> source point
    # d -> destination (receiving point)
    s = np.asarray(s)
    d = np.asarray(d)
    return np.linalg.norm(d - s)

def free_space_transfer_function(r_s, r_r, k) -> np.array:
    # r_s -> matrix of source points, shape (Ns, 2)
    # r_r -> matrix of recieving points, shape (Nr, 2)
    # k -> wavenumber
    # returns g -> matrix of floats, shape (Nr, Ns)
    g = np.zeros((len(r_r), len(r_s)), dtype=complex)
    for i, r in enumerate(r_r):
        for j, s in enumerate(r_s):
            d = euclidean_distance(s, r)
            # Check for eq 39
            if j == 2 and i == 0:
                check_d = d
            numerator = -np.exp(1j * k * d)
            denominator = 4 * np.pi * d
            g[i, j] = np.round(numerator / denominator, 5) # rounding here is questionable, but I need it for the checks
    return g, check_d

def sum_rule(g:np.array(complex)) -> float:
    S = np.asarray(0., dtype=np.float64)
    for i in g:
        for j in i:
            S += np.abs(j)**2
    return S

def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")

In [4]:
s = np.asarray([(0, i * (wavelength / 2)) for i in range(Ns)])
d = np.asarray([(Lz, i * (wavelength / 2)) for i in range(Nr)])
Gsr, check_d = free_space_transfer_function(s, d, k)
gsr = np.round(-4 * np.pi * Lz, 2)

In [8]:
matprint(Gsr)

      -0.01592+0j  -0.01564-0.00247j   -0.01268-0.0091j  -0.00284-0.01498j   0.01109-0.00976j   0.01201+0.00764j  -0.00665+0.01192j  -0.01039-0.00788j    0.0102-0.00711j  
-0.01564-0.00247j        -0.01592+0j  -0.01564-0.00247j   -0.01268-0.0091j  -0.00284-0.01498j   0.01109-0.00976j   0.01201+0.00764j  -0.00665+0.01192j  -0.01039-0.00788j  
 -0.01268-0.0091j  -0.01564-0.00247j        -0.01592+0j  -0.01564-0.00247j   -0.01268-0.0091j  -0.00284-0.01498j   0.01109-0.00976j   0.01201+0.00764j  -0.00665+0.01192j  
-0.00284-0.01498j   -0.01268-0.0091j  -0.01564-0.00247j        -0.01592+0j  -0.01564-0.00247j   -0.01268-0.0091j  -0.00284-0.01498j   0.01109-0.00976j   0.01201+0.00764j  
 0.01109-0.00976j  -0.00284-0.01498j   -0.01268-0.0091j  -0.01564-0.00247j        -0.01592+0j  -0.01564-0.00247j   -0.01268-0.0091j  -0.00284-0.01498j   0.01109-0.00976j  
 0.01201+0.00764j   0.01109-0.00976j  -0.00284-0.01498j   -0.01268-0.0091j  -0.01564-0.00247j        -0.01592+0j  -0.01564-0.00247j   -0.012

In [5]:
S = sum_rule(Gsr)

In [9]:
# eq 51 check
print(np.allclose(S, 72.65 / (gsr**2), atol=1e-6))

True


In [10]:
gsr_Gsrd_Gsr = np.round(gsr**2 * np.matmul(np.matrix.getH(Gsr), Gsr), 2)
matprint(gsr_Gsrd_Gsr)
print(gsr_Gsrd_Gsr.shape)

   7.55+0j   3.95-4.72j   0.03-2.32j   0.12+1.16j    0.6+1.63j    0.08+0.1j   0.18-1.11j   0.49-0.54j    -0.35-0j  
3.95+4.72j      7.93+0j   5.02-3.67j   0.95-1.72j  -0.74+1.22j  -0.93+1.36j   0.06-0.02j      1.15-0j  0.49+0.54j  
0.03+2.32j   5.02+3.67j      8.22+0j    5.8-2.33j   1.48-0.89j  -1.46+0.53j     -1.61-0j   0.06+0.02j  0.18+1.11j  
0.12-1.16j   0.95+1.72j    5.8+2.33j       8.4+0j    6.21-0.8j      1.64+0j  -1.46-0.53j  -0.93-1.36j   0.08-0.1j  
 0.6-1.63j  -0.74-1.22j   1.48+0.89j    6.21+0.8j      8.46+0j    6.21+0.8j   1.48+0.89j  -0.74-1.22j   0.6-1.63j  
 0.08-0.1j  -0.93-1.36j  -1.46-0.53j      1.64+0j    6.21-0.8j       8.4+0j    5.8+2.33j   0.95+1.72j  0.12-1.16j  
0.18+1.11j   0.06+0.02j     -1.61+0j  -1.46+0.53j   1.48-0.89j    5.8-2.33j      8.22+0j   5.02+3.67j  0.03+2.32j  
0.49+0.54j      1.15+0j   0.06-0.02j  -0.93+1.36j  -0.74+1.22j   0.95-1.72j   5.02-3.67j      7.93+0j  3.95+4.72j  
  -0.35+0j   0.49-0.54j   0.18-1.11j    0.08+0.1j    0.6+1.63j   0.12+1.

In [11]:

from scipy.linalg import eigh

Gsr_Gsrd = np.matmul(np.matrix.getH(Gsr), Gsr)
eig_vals, eig_vect = np.linalg.eigh(Gsr_Gsrd)
eig_vect = np.round(np.flip(eig_vect, axis=1), 2)
eig_vals = np.round(np.flip(eig_vals), 5)
print(eig_vals.shape)
print(eig_vals)
print(eig_vals * (gsr**2))
print(np.sum(eig_vals * (gsr**2)))

(9,)
[5.25e-03 5.17e-03 4.84e-03 2.64e-03 4.80e-04 3.00e-05 0.00e+00 0.00e+00
 0.00e+00]
[20.72494672 20.40913801 19.10642708 10.4216875   1.89485227  0.11842827
  0.          0.          0.        ]
72.67547984899998


In [12]:
eig_vect.shape

(9, 9)

In [13]:
# Choose the central phase to be zero, normalize by it
def normalize_phase(eigenvector):
    # Choose middle index (zero-based)
    mid_idx = len(eigenvector) // 2
    # Extract phase of the middle entry
    phase = np.angle(eigenvector[mid_idx])
    # Normalize eigenvector by this phase
    return eigenvector * np.exp(-1j * phase)


# Normalize each eigenvector's phase
normalized_eigenvectors = np.column_stack([
    normalize_phase(eig_vect[:, j])
    for j in range(eig_vect.shape[1])
])

In [14]:
x = np.linspace(0.2, Lz+0.2, 200)
y = np.linspace(-dy, dy * Ns, 200)
xx, yy = np.meshgrid(x,y)

points = np.stack([xx.ravel(), yy.ravel()], axis=-1)

In [15]:
distance = []
for p in points:
    distance.append(euclidean_distance(s[3], p))
distance = np.stack(distance)
print(distance.shape)
distance = np.reshape(distance, xx.shape)
print(distance.shape)


(40000,)
(200, 200)


In [16]:
values = []

for m in range(0, 9):
    temp = []
    for p in points:
        h = normalized_eigenvectors[:, m]
        val = 0
        for i, source in enumerate(s):
            distance = euclidean_distance(source, p)
            numerator = np.exp(1j * k * distance) * h[i]
            val += numerator / distance
        val *= (-1 / (4 * np.pi))
        val *= np.sqrt(p[0])
        temp.append(val)
    values.append(temp)
values = np.asarray(values)

print(values.shape)
values = np.asarray([np.reshape(i, xx.shape) for i in values])
print(values.shape)

(9, 40000)
(9, 200, 200)


In [ ]:
# Convert to arrays if they aren't already
source_points = np.asarray(s)
dest_points = np.asarray(points)
normalized_eigenvectors = np.asarray(normalized_eigenvectors)

# Precompute all pairwise distances: shape (num_dest, num_source)
distances = cdist(dest_points, source_points)  # much faster than nested loops

# Precompute Green's function kernel (excluding eigenvector weighting)
greens_kernel = np.exp(1j * k * distances) / distances  # shape: (num_dest, num_source)

# Apply the integral and eigenvector projection
# Transpose normalized_eigenvectors to shape (num_modes, num_source)
projected = greens_kernel @ normalized_eigenvectors  # shape: (num_dest, num_modes)

# Multiply by constants and reshape to match original output (num_modes, num_dest)
scale_factor = (-1 / (4 * np.pi)) * np.sqrt(dest_points[:,0])
values_temp = scale_factor * projected.T  # shape: (num_modes, num_dest)
values_temp = np.asarray([np.reshape(i, xx.shape) for i in values_temp])


In [ ]:
print(dest_points.shape)

In [ ]:
for m in values:
    print(m.shape)

In [ ]:
print(values_temp.shape)

In [ ]:
for v, v_ in zip(values, values_temp):
    print(np.allclose(v, v_))

In [ ]:
for i,m in enumerate(values_temp):
    fig, ax = plt.subplots(2,2, figsize=(10,10))
    
    
    ax[0][0].pcolormesh(xx, yy, m.real, cmap='jet')
    ax[0][0].scatter(s[:, 0], s[:, 1], color='black')
    ax[0][0].scatter(d[:, 0], d[:, 1], color='black')
    ax[0][0].set_title("Real")
    ax[0][0].set_xlabel(r"Z [$\lambda$]")
    ax[0][0].set_ylabel(r"Y [$\lambda$]")

    
    ax[0][1].pcolormesh(xx, yy,m.imag, cmap='jet')
    ax[0][1].scatter(s[:, 0], s[:, 1], color='black')
    ax[0][1].scatter(d[:, 0], d[:, 1], color='black')
    ax[0][1].set_title("Imaginary")
    ax[0][1].set_xlabel(r"Z [$\lambda$]")
    ax[0][1].set_ylabel(r"Y [$\lambda$]")
    
    ax[1][0].pcolormesh(xx, yy, np.abs(m), cmap='jet')
    ax[1][0].scatter(s[:, 0], s[:, 1], color='black')
    ax[1][0].scatter(d[:, 0], d[:, 1], color='black')
    ax[1][0].set_title("Magnitude")
    ax[1][0].set_xlabel(r"Z [$\lambda$]")
    ax[1][0].set_ylabel(r"Y [$\lambda$]")
    
    ax[1][1].pcolormesh(xx, yy, np.angle(m), cmap='hsv')
    ax[1][1].scatter(s[:, 0], s[:, 1], color='black')
    ax[1][1].scatter(d[:, 0], d[:, 1], color='black')
    ax[1][1].set_title("Phase")
    ax[1][1].set_xlabel(r"Z [$\lambda$]")
    ax[1][1].set_ylabel(r"Y [$\lambda$]")
    
    fig.suptitle("Mode {}".format(i))
    plt.tight_layout()
    fig.savefig("SS5_1_modes/mode_{:03d}.png".format(i), dpi=300)

## Let's get the source complex amplitudes

In [ ]:
plt.scatter(range(0,9), np.angle(normalized_eigenvectors[:,1]))

In [ ]:
np.angle(eig_vect[:,1][4])